<a href="https://colab.research.google.com/github/saipavan70/gen_ai/blob/main/GenAi_bert_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [ ]:
# Specify the pre-trained model to use: BERT-base-cased
model_name = "bert-base-cased"

# Instantiate the tokenizer and model for the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Get the mask token from the tokenizer
mask = tokenizer.mask_token
print(mask)

[MASK]


In [ ]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"I want {mask}to play {mask} online game ."
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)

['I', 'want', '[MASK]', 'to', 'play', '[MASK]', 'online', 'game', '.']


In [ ]:
# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')
print(encoded_inputs)

{'input_ids': tensor([[ 101,  146, 1328, 1106, 1505,  103, 1342,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# Get the model's output for the input tensors
outputs = model(**encoded_inputs)
print(outputs)

MaskedLMOutput(loss=None, logits=tensor([[[ -7.4441,  -7.3595,  -7.4866,  ...,  -6.2332,  -6.0051,  -6.3782],
         [ -7.9406,  -8.2047,  -7.9583,  ...,  -6.5737,  -6.4586,  -6.9132],
         [-10.2686, -10.0274, -10.0415,  ...,  -5.8391,  -6.1988,  -7.2037],
         ...,
         [ -9.3711,  -9.7368,  -9.7700,  ...,  -6.6204,  -7.1579,  -9.6350],
         [-10.9867, -10.9860, -10.7428,  ...,  -7.8914,  -9.0010,  -9.3707],
         [-12.6182, -13.1229, -12.7733,  ...,  -9.5278, -10.9504, -11.4559]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [ ]:
# Detach the logits from the model's output and convert them to numpy arrays
logits = outputs.logits.detach().numpy()[0]
logits.shape

(9, 28996)

In [ ]:
len(tokens)

7

In [ ]:
# Extract the logits for the mask token
mask_logits = logits[tokens.index(mask) + 1]
print(mask_logits)

[-8.048591  -7.969764  -7.7299833 ... -6.4922104 -6.4843874 -7.6569347]


In [ ]:
# Calculate the confidence scores for each possible token using softmax
confidence_scores = softmax(mask_logits)
print(confidence_scores)

[7.56993426e-11 8.19079735e-11 1.04102602e-10 ... 3.58938407e-10
 3.61757013e-10 1.11991666e-10]


In [ ]:
confidence_scores.sum()

1.0000001

In [ ]:
# Print the top 5 predicted tokens and their confidence scores
for i in np.argsort(confidence_scores)[::-1][:25]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # Print the predicted sentence with the mask token replaced by the predicted token, and the confidence score
    print(sentence.replace(mask, pred_token), score)

I want thisto play this online game . 0.48249835
I want theto play the online game . 0.20590052
I want thatto play that online game . 0.12497481
I want ato play a online game . 0.093214475
I want myto play my online game . 0.016889594
I want histo play his online game . 0.015109516
I want yourto play your online game . 0.014637972
I want herto play her online game . 0.012521489
I want anotherto play another online game . 0.007279805
I want theirto play their online game . 0.00629853
I want ourto play our online game . 0.0048808297
I want everyto play every online game . 0.0036181384
I want anyto play any online game . 0.0031221109
I want someto play some online game . 0.0030805003
I want oneto play one online game . 0.00093708886
I want itsto play its online game . 0.00056375586
I want noto play no online game . 0.00048418745
I want whateverto play whatever online game . 0.00045337653
I want itto play it online game . 0.00024104952
I want theseto play these online game . 0.00014631984


In [ ]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

# Specify the pre-trained model to use: BERT-base-cased
model_name = "bert-base-cased"

# Instantiate the tokenizer and model for the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Get the mask tokens from the tokenizer
mask1 = tokenizer.mask_token
mask2 = tokenizer.mask_token

# Create a sentence with two mask tokens to be filled in by the model
sentence = f"I want to {mask1} pizza and {mask2} for tonight."

# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)

# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')

# Get the model's output for the input tensors
outputs = model(**encoded_inputs)

# Detach the logits from the model's output and convert them to numpy arrays
logits = outputs.logits.detach().numpy()[0]

# Extract the logits for the first mask token
mask1_logits = logits[tokens.index(mask1) + 1]

# Extract the logits for the second mask token
mask2_logits = logits[tokens.index(mask2) + 1]

# Calculate the confidence scores for each possible token using softmax for the first mask token
confidence_scores1 = softmax(mask1_logits)

# Calculate the confidence scores for each possible token using softmax for the second mask token
confidence_scores2 = softmax(mask2_logits)

# Print the top 5 predicted tokens and their confidence scores for the first mask token
print("Top predictions for the first mask:")
for i in np.argsort(confidence_scores1)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores1[i]
    print(sentence.replace(mask1, pred_token), score)

